In [1]:
import codecs,glob,re
import random as rd
import itertools as it

In [2]:
nbLiens=3
rd.seed(2464)
nomRepCours="/Users/gilles/Downloads/"
nomRepCours="/Volumes/BroadExt/Gilles/Downloads/"
#nomFichierEntreeXML=nomRepCours+"quiz-LNS6E812-EcriScol-20200505-0952.xml"
nomFichierEntreeXML=nomRepCours+"quiz-LNS6E812-EcriScol-20200505-1656.xml"
#nomFichierLiens="/Users/gilles/pCloud Drive/FOD/GB/SdlEcole/LiensEcoleVelo.txt"
nomFichierLiens="/Users/gilles/pCloud Drive/FOD/GB/Cours/L3-SdlEcole/LiensClasseVerte.txt"

In [3]:
with codecs.open(nomFichierEntreeXML,"r",encoding="utf8") as input:
    xmlQuestions=[l.strip() for l in input.readlines()]

In [4]:
def getTemplate(xmlLines,qType="essay"):
    header=[]
    questionTemplate=[]
    trailer=[]
    boolTemplate=False
    boolTrailer=False
    for xmlLine in xmlLines:
        m=re.search(ur'<question type="essay">',xmlLine)
        if m:
            questionTemplate.append(xmlLine)
            boolTemplate=True
        elif ur"</question>" in xmlLine and boolTemplate:
            questionTemplate.append(xmlLine)
            boolTemplate=False
            boolTrailer=True
        elif boolTemplate:
            questionTemplate.append(xmlLine)
        elif boolTrailer:
            trailer.append(xmlLine)
        else:
            header.append(xmlLine)
    return header, questionTemplate, trailer

In [5]:
header,templateLines,trailer=getTemplate(xmlQuestions)

In [6]:
templateLines

[u'<question type="essay">',
 u'<name>',
 u'<text>\xc9valuations de E01, E02 et E03</text>',
 u'</name>',
 u'<questiontext format="html">',
 u'<text><![CDATA[<p>\xc9valuez les trois productions suivantes (<a href="url1" target="_blank">Camille</a>, <a href="url2" target="_blank">Dominique</a>, <a href="url3" target="_blank">Yannick</a>) pour l\'activit\xe9 \xab J\'\xe9cris un texte argumentatif \xbb pr\xe9par\xe9e en classe de CE2-CM1 en vous aidant cet <a href="https://mycore.core-cloud.net/index.php/s/QRUNjhby3FqAcz0" target="_blank">extrait</a> du compte-rendu de la pr\xe9paration (ne tenez pas compte des arguments qui concernent aller \xe0 l\'\xe9cole en v\xe9lo).</p><p>Vous \xe9tablirez un profil compar\xe9 pour les 3 productions sur des \xe9chelles \xe0 5 valeurs (pas de demi-barreaux) en justifiant vos jugements pour toutes les productions pour chaque \xe9l\xe9ment pertinent que vous avez inclus dans le profil (minimum 5 \xe9l\xe9ments, maximum 8).</p><p>Vous d\xe9poserez votre 

In [7]:
with codecs.open(nomFichierLiens,"r",encoding="utf8") as input:
    liens=[l.strip() for l in input.readlines()]
#liens

In [8]:
def newQuestion(templateLines,(a,b,c)):
    result=[]
    for line in templateLines:
        m=re.search(ur"Évaluations de E01, E02 et E03",line)
        if m:
            line=line.replace(u"Évaluations de E01, E02 et E03",u"Évaluations de E%02d, E%02d et E%02d"%(a+1,b+1,c+1))
        line=line.replace("url1",liens[a])
        line=line.replace("url2",liens[b])
        line=line.replace("url3",liens[c])
        result.append(line)
    return result

In [9]:
def newQuestions(templateLines,nbQuestions):
    result=[]
    combinations=rd.sample(list(it.combinations(range(len(liens)),nbLiens)),nbQuestions)
    for c in combinations:
        question=newQuestion(templateLines,c)
        result.extend(question)
    return result

In [10]:
sortieXML=header+newQuestions(templateLines,43)+trailer
print "\n".join(sortieXML)

<?xml version="1.0" encoding="UTF-8"?>
<quiz>
<!-- question: 0  -->
<question type="category">
<category>
<text>$course$/top/EcriScol/Bibli</text>
</category>
<info format="html">
<text></text>
</info>
<idnumber></idnumber>
</question>

<!-- question: 24967848  -->
<question type="essay">
<name>
<text>Évaluations de E03, E07 et E10</text>
</name>
<questiontext format="html">
<text><![CDATA[<p>Évaluez les trois productions suivantes (<a href="https://my.pcloud.com/publink/show?code=XZyyslkZ8xJjyw3QmjbbFbdk5PRDbmNKDJmk" target="_blank">Camille</a>, <a href="https://my.pcloud.com/publink/show?code=XZ7kslkZyQxfMQUkt5f5E2M8MNiYlLyj96MX" target="_blank">Dominique</a>, <a href="https://my.pcloud.com/publink/show?code=XZLkslkZilSIIlFqy8uW7KOxT7QRDLtBhWny" target="_blank">Yannick</a>) pour l'activité « J'écris un texte argumentatif » préparée en classe de CE2-CM1 en vous aidant cet <a href="https://mycore.core-cloud.net/index.php/s/QRUNjhby3FqAcz0" target="_blank">extrait</a> du compte-rendu de

In [11]:
with codecs.open(nomFichierEntreeXML.replace(".xml","-VAR.xml"),"w",encoding="utf8") as outFile:
    outFile.write("\n".join(sortieXML))